In [10]:
import numpy as np
from matplotlib.image import imread

ModuleNotFoundError: No module named 'cv2'

In [11]:
np.convolve([1,2,3,4,5],[1,2])

array([ 1,  4,  7, 10, 13, 10])

In [12]:
a = np.array([[0, 1, 0], [0, 1, 0], [0, 1, 0]])

In [9]:
b = cv2.imread("H_test.jpg")

NameError: name 'cv2' is not defined